<a href="https://colab.research.google.com/github/Noob919/IoT_Security_Project/blob/main/MultiClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

In [3]:
import os
import dask.dataframe as dd

# Set the dataset directory
DATASET_DIRECTORY = '/content/drive/MyDrive/Data/'

# Importing Dataset

df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
df_sets.sort()
training_sets = df_sets[:int(len(df_sets) * 0.8)]
test_sets = df_sets[int(len(df_sets) * 0.8):]

# Create a Dask DataFrame for training data
training_dataframes = [dd.read_csv(DATASET_DIRECTORY + train_set) for train_set in training_sets]

# Concatenate all Dask DataFrames into a single training Dask DataFrame
training_data = dd.concat(training_dataframes, axis=0)

# Compute summary statistics
summary_stats = training_data.describe().compute()
summary_stats

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight
count,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,...,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07,3.634642e+07
mean,5.776241e+00,7.669673e+04,9.066452e+00,6.635099e+01,9.055259e+03,9.055259e+03,5.906723e-06,8.656754e-02,2.073098e-01,9.049695e-02,...,1.246705e+02,3.333304e+01,1.246947e+02,8.318599e+07,9.498651e+00,1.312189e+01,4.710645e+01,3.074485e+04,9.644400e-02,1.415166e+02
std,2.825513e+02,4.611977e+05,8.945459e+00,1.402003e+01,9.943559e+04,9.943559e+04,8.097916e-03,2.812003e-01,4.053794e-01,2.868924e-01,...,2.410143e+02,1.604007e+02,2.415784e+02,1.704543e+07,8.190662e-01,8.628687e+00,2.268605e+02,3.258548e+05,2.330306e-01,2.106605e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.200000e+01,0.000000e+00,4.200000e+01,0.000000e+00,1.000000e+00,9.165151e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,0.000000e+00,5.400000e+01,6.000000e+00,6.400000e+01,2.139973e+00,2.139973e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.000000e+01,0.000000e+00,5.000000e+01,8.307200e+07,9.500000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.415500e+02
50%,0.000000e+00,5.400000e+01,6.000000e+00,6.400000e+01,1.606355e+01,1.606355e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.400000e+01,0.000000e+00,5.400000e+01,8.312463e+07,9.500000e+00,1.039230e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.415500e+02
75%,1.078261e-01,4.161725e+02,1.524000e+01,6.400000e+01,1.230812e+02,1.230812e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.405734e+01,3.955494e-01,5.406000e+01,8.334402e+07,9.500000e+00,1.039739e+01,5.303656e-01,1.516787e+00,8.000000e-02,1.415500e+02
max,2.629164e+05,9.907148e+06,4.700000e+01,2.550000e+02,8.388608e+06,8.388608e+06,2.971522e+01,1.000000e+00,1.000000e+00,1.000000e+00,...,1.160047e+04,1.238524e+04,1.309800e+04,1.676394e+08,1.500000e+01,1.453904e+02,1.755127e+04,1.549022e+08,1.000000e+00,2.446000e+02


In [ ]:
# List of label values you want to filter on
label_values_to_select = [
    'DDoS-RSTFINFlood',
    'DoS-TCP_Flood',
    'DDoS-ICMP_Flood',
    'DoS-UDP_Flood',
    'DoS-SYN_Flood',
    'DDoS-SynonymousIP_Flood',
    'DDoS-SYN_Flood',
    'DDoS-PSHACK_Flood',
    'DDoS-TCP_Flood',
    'DDoS-UDP_Flood',
    'DDoS-ACK_Fragmentation',
    'DoS-HTTP_Flood',
    'DDoS-ICMP_Fragmentation',
    'DDoS-UDP_Fragmentation',
    'DDoS-HTTP_Flood',
    'DDoS-SlowLoris'
]

# Create an empty list to store selected rows
selected_rows = []

# Loop through each label value and select the first 300 rows for each
for label_value in label_values_to_select:
    label_rows = training_data[training_data['label'] == label_value].head(800)
    selected_rows.append(label_rows)

# Concatenate the selected rows into a single DataFrame
selected_data = dd.concat(selected_rows, axis=0)
selected_data.compute()

# Logistic Regression

In [136]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Logistic Regression)
multiclass_classifier = LogisticRegression(max_iter=10000, random_state=42)

# Fit the multiclass classifier on the scaled training data
multiclass_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'DDos']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Fit and transform the training data
final_x_scaled = numeric_transformer.fit_transform(final_X)

# Predict on the scaled test data
multiclass_predictions = multiclass_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")


['DDoS-ICMP_Flood' 'DDoS-ACK_Fragmentation' 'DDoS-TCP_Flood' ...
 'DDoS-RSTFINFlood' 'DDoS-SynonymousIP_Flood' 'DDoS-TCP_Flood']
Multiclass Classification Accuracy: 0.70
Multiclass Classification F1 Score: 0.64
Multiclass Classification Recall Score: 0.70


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Random Forest

In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Random Forest)
multiclass_classifier = RandomForestClassifier(n_estimators=800, random_state=42)

# Fit the multiclass classifier on the scaled training data
multiclass_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'DDos']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Fit and transform the training data
final_x_scaled = numeric_transformer.fit_transform(final_X)

# Predict on the scaled test data
multiclass_predictions = multiclass_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(y_test_multiclass, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(y_test_multiclass, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_recall = recall_score(y_test_multiclass, multiclass_predictions, average='weighted')
print(f"Multiclass Classification recall Score: {multiclass_f1_score:.2f}")

['DDoS-ICMP_Flood' 'DoS-HTTP_Flood' 'DDoS-TCP_Flood' ...
 'DDoS-RSTFINFlood' 'DDoS-SYN_Flood' 'DDoS-TCP_Flood']
Multiclass Classification Accuracy: 0.61
Multiclass Classification F1 Score: 0.54
Multiclass Classification recall Score: 0.54


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SVM

In [138]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Support Vector Machine)
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)

# Fit the SVM classifier on the scaled training data
svm_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'DDos']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Transform the test data using the same scaler
final_x_scaled = numeric_transformer.transform(final_X)

# Predict on the scaled test data
multiclass_predictions = svm_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")

['DDoS-ICMP_Flood' 'DDoS-ACK_Fragmentation' 'DDoS-TCP_Flood' ...
 'DDoS-RSTFINFlood' 'DoS-SYN_Flood' 'DoS-TCP_Flood']
Multiclass Classification Accuracy: 0.81
Multiclass Classification F1 Score: 0.78
Multiclass Classification Recall Score: 0.81


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Decison Tree

In [139]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Decision Tree)
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Fit the Decision Tree classifier on the scaled training data
decision_tree_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'DDos']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Transform the test data using the same scaler
final_x_scaled = numeric_transformer.transform(final_X)

# Predict on the scaled test data
multiclass_predictions = decision_tree_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")

['DDoS-ICMP_Flood' 'DDoS-ACK_Fragmentation' 'DDoS-TCP_Flood' ...
 'DDoS-RSTFINFlood' 'DoS-SYN_Flood' 'DoS-TCP_Flood']
Multiclass Classification Accuracy: 0.92
Multiclass Classification F1 Score: 0.89
Multiclass Classification Recall Score: 0.92


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **MLP**

In [140]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (Multi-Layer Perceptron)
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

# Fit the MLP classifier on the scaled training data
mlp_classifier.fit(X_train_scaled, y_multiclass)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'DDos']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Transform the test data using the same scaler
final_x_scaled = numeric_transformer.transform(final_X)

# Predict on the scaled test data
multiclass_predictions = mlp_classifier.predict(final_x_scaled)
print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")

['DDoS-ICMP_Flood' 'DDoS-ACK_Fragmentation' 'DDoS-TCP_Flood' ...
 'DDoS-RSTFINFlood' 'DoS-SYN_Flood' 'DoS-TCP_Flood']
Multiclass Classification Accuracy: 0.92
Multiclass Classification F1 Score: 0.88
Multiclass Classification Recall Score: 0.92


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **XGBoost**

In [141]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score
import pandas as pd

# Load your data (replace this with your dataset)
# For demonstration purposes, we'll use the Iris dataset.

X = selected_data.drop(columns=['label']).compute()
y_multiclass = selected_data['label'].compute()

# Encode the target labels using LabelEncoder
label_encoder = LabelEncoder()
y_multiclass_encoded = label_encoder.fit_transform(y_multiclass)

# Define the transformer for feature scaling
numeric_transformer = StandardScaler()

# Fit and transform the training data
X_train_scaled = numeric_transformer.fit_transform(X)

# Create a multiclass classifier (XGBoost)
xgb_classifier = xgb.XGBClassifier(random_state=42)

# Fit the XGBoost classifier on the scaled training data
xgb_classifier.fit(X_train_scaled, y_multiclass_encoded)

df = pd.read_csv("/content/pred_df.csv")

final = df[df['Binary Prediction'] == 'DDos']

final_X = final.drop(columns=['z_test', 'new_label', 'Binary Prediction'])
final_y = final['z_test']

# Transform the test data using the same scaler
final_x_scaled = numeric_transformer.transform(final_X)

# Predict on the scaled test data
multiclass_predictions_encoded = xgb_classifier.predict(final_x_scaled)

# Inverse transform the predictions to get original labels
multiclass_predictions = label_encoder.inverse_transform(multiclass_predictions_encoded)

print(multiclass_predictions)

# Evaluate the overall accuracy for multiclass classification
multiclass_accuracy = accuracy_score(final_y, multiclass_predictions)
print(f"Multiclass Classification Accuracy: {multiclass_accuracy:.2f}")

# Calculate the F1 score for multiclass classification
multiclass_f1_score = f1_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification F1 Score: {multiclass_f1_score:.2f}")

# Calculate the recall score for multiclass classification
multiclass_recall = recall_score(final_y, multiclass_predictions, average='weighted')
print(f"Multiclass Classification Recall Score: {multiclass_recall:.2f}")

['DDoS-ICMP_Flood' 'DoS-HTTP_Flood' 'DDoS-TCP_Flood' ...
 'DDoS-RSTFINFlood' 'DoS-SYN_Flood' 'DoS-TCP_Flood']
Multiclass Classification Accuracy: 0.92
Multiclass Classification F1 Score: 0.89
Multiclass Classification Recall Score: 0.92


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
